<a href="https://colab.research.google.com/github/sudiptoghosh8/3d_image_processing/blob/main/NLP_TASK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/drive/MyDrive/nlp/DATASET - DATASET.csv")
text_data = data["input_address"]


In [4]:
text_data = text_data.str.lower()


In [5]:
text_data = text_data.str.replace(r"[^\w\s]", "")
text_data = text_data.str.strip()


<ipython-input-5-239fcb984732>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  text_data = text_data.str.replace(r"[^\w\s]", "")


In [6]:
!pip install nltk

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
tokenized_data = text_data.apply(nltk.word_tokenize)

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
stop_words = stopwords.words("english","bengali")
tokenized_data = tokenized_data.apply(lambda tokens: [token for token in tokens if token not in stop_words])


In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

tokenized_data = tokenized_data.apply(lambda tokens: [stemmer.stem(token) for token in tokens])

tokenized_data = tokenized_data.apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

In [13]:
data["preprocessed_text"] = tokenized_data.apply(lambda tokens: " ".join(tokens))
data.to_csv("preprocessed_data.csv", index=False)


In [14]:
data

,input_address,preprocessed_text
0,"House -122\/4\/A, Ayesha Monjil(Level-6), Lal ...",hous 1224a ayesha monjillevel6 lal mia goli be...
1,"184\/6 east goran,road no 8, near shahi moshji...",1846 east goranroad 8 near shahi moshjid khilg...
2,"386\/A (Euro Garden),Near Gulbagh jame mosjid,...",386a euro gardennear gulbagh jame mosjidgulbag...
3,"House 306\/C, Road 1, Shovon Shoroni, Middle P...",hous 306c road 1 shovon shoroni middl pirerbag...
4,"Shahjalal tower 2, beside samsul haque khan Co...",shahjal tower 2 besid samsul haqu khan colleg ...
...,...,...
49137,"address: 56\/1, green road, kathalbagan bazare...",address 561 green road kathalbagan bazar golir...
49138,"Thonthonia, sufipara, opposite dhaka bus stand...",thonthonia sufipara opposit dhaka bu standbogu...
49139,"house-38 & 40, road-7, Block-E, behind evercar...",house38 40 road7 block behind evercar bashundh...
49140,"holding nub 28\/35 ,post office colony ,near i...",hold nub 2835 post offic coloni near ibn sina ...


In [15]:


if "preprocessed_text" not in data.columns:
    raise ValueError("Column 'preprocessed_text' not found in the CSV file.")


def categorize_text(text):
    keywords = {
        "Education": ["school", "university", "college", "library", "course", "training", "education"],
        "Religious Place": ["mosjid", "temple", "church", "pagoda", "cathedral", "shrine", "monastery","mosquie","moshjid","mondir"],
        "Commercial": ["shop", "store", "market", "plaza", "mall", "restaurant", "cafe", "hotel", "office"],
        "Healthcare": ["hospital", "clinic", "pharmacy", "doctor", "nurse", "medicine", "healthcare"],
        "Recreation": ["park", "stadium", "museum", "theater", "cinema", "zoo", "amusement", "sport", "play"],
        "Office": ["office", "building", "company", "business", "work", "headquarters", "corporation"],
        "Bank": ["bank", "atm", "finance", "money", "loan", "account", "investment"],
        "Food": ["restaurant", "cafe", "food", "drink", "bakery", "cuisine", "kitchen", "dining"],
        "Hotel": ["hotel", "lodge", "motel", "accommodation", "hostel", "resort", "guesthouse"],
        "Residential": ["house", "apartment", "flat", "home", "building", "residential", "property"],
        "Utility": ["electricity", "water", "gas", "waste", "garbage", "sanitation", "infrastructure"],
        "Fuel": ["petrol", "diesel", "gasoline", "fuel", "energy", "station", "refinery"],
        "Agriculture": ["farm", "land", "crop", "livestock", "farming", "agriculture", "harvest"],
        "Construction": ["construction", "building", "project", "development", "architecture", "engineering", "infrastructure"]
    }


    multiple_categories = []
    for category, category_keywords in keywords.items():
        if any(keyword in text for keyword in category_keywords):
            multiple_categories.append(category)


    if len(multiple_categories) > 1:

        category = multiple_categories[0]
    else:
        category = multiple_categories[0] if multiple_categories else "Other"

    return category


data["category"] = data["preprocessed_text"].apply(categorize_text)


print(data)
data.to_csv("labeled_dataset.csv", index=False)


                                           input_address  \
0      House -122\/4\/A, Ayesha Monjil(Level-6), Lal ...   
1      184\/6 east goran,road no 8, near shahi moshji...   
2      386\/A (Euro Garden),Near Gulbagh jame mosjid,...   
3      House 306\/C, Road 1, Shovon Shoroni, Middle P...   
4      Shahjalal tower 2, beside samsul haque khan Co...   
...                                                  ...   
49137  address: 56\/1, green road, kathalbagan bazare...   
49138  Thonthonia, sufipara, opposite dhaka bus stand...   
49139  house-38 & 40, road-7, Block-E, behind evercar...   
49140  holding nub 28\/35 ,post office colony ,near i...   
49141  Monjil samir tawer,Dokkhin mollartek Ashkona, ...   

                                       preprocessed_text         category  
0      hous 1224a ayesha monjillevel6 lal mia goli be...            Other  
1      1846 east goranroad 8 near shahi moshjid khilg...  Religious Place  
2      386a euro gardennear gulbagh jame mosjidgulb

In [16]:
data

,input_address,preprocessed_text,category
0,"House -122\/4\/A, Ayesha Monjil(Level-6), Lal ...",hous 1224a ayesha monjillevel6 lal mia goli be...,Other
1,"184\/6 east goran,road no 8, near shahi moshji...",1846 east goranroad 8 near shahi moshjid khilg...,Religious Place
2,"386\/A (Euro Garden),Near Gulbagh jame mosjid,...",386a euro gardennear gulbagh jame mosjidgulbag...,Religious Place
3,"House 306\/C, Road 1, Shovon Shoroni, Middle P...",hous 306c road 1 shovon shoroni middl pirerbag...,Other
4,"Shahjalal tower 2, beside samsul haque khan Co...",shahjal tower 2 besid samsul haqu khan colleg ...,Other
...,...,...,...
49137,"address: 56\/1, green road, kathalbagan bazare...",address 561 green road kathalbagan bazar golir...,Other
49138,"Thonthonia, sufipara, opposite dhaka bus stand...",thonthonia sufipara opposit dhaka bu standbogu...,Other
49139,"house-38 & 40, road-7, Block-E, behind evercar...",house38 40 road7 block behind evercar bashundh...,Residential
49140,"holding nub 28\/35 ,post office colony ,near i...",hold nub 2835 post offic coloni near ibn sina ...,Healthcare


In [43]:
X = data["preprocessed_text"]
y = data["category"]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [45]:
from sklearn.svm import SVC

In [46]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_vec, y_train)


y_pred = svm_classifier.predict(X_test_vec)


accuracy = accuracy_score(y_test, y_pred)
print("SVM Classifier Accuracy:", accuracy)

SVM Classifier Accuracy: 0.9051753374482805


In [47]:
def preprocess_address(address):
    address = address.lower()
    address = re.sub(r"[^\w\s]", "", address)
    address = address.strip()
    tokens = nltk.word_tokenize(address)
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_address = " ".join(tokens)
    return preprocessed_address

In [48]:
def predict_category(address):

    preprocessed_address = preprocess_address(address)

    address_vec = vectorizer.transform([preprocessed_address])

    category = svm_classifier.predict(address_vec)[0]

    return category

In [49]:
input_address = "West Agargaon near science Museum Bnp Bazar new Parliament complex,Dhaka 1207, flat 201, building C-2"

# Predict category
predicted_category = predict_category(input_address)
print("Predicted Category:", predicted_category)


Predicted Category: Recreation


In [ ]:
while(1):
  input_address = input("Enter a address: ")

  # Predict category
  predicted_category = predict_category(input_address)
  print("Predicted Category:", predicted_category)

Enter a address: West Agargaon near science Museum Bnp Bazar new Parliament complex,Dhaka 1207, flat 201, building C-2
Predicted Category: Recreation
Enter a address: West Agargaon near science Museum Bnp Bazar new Parliament complex,Dhaka 1207, flat 201, building C-2
Predicted Category: Recreation
